<style>
    img {
        display: block;
        margin-left: auto;
        margin-right: auto;
    }
</style>


# Predicting FOMC Impact on Market
Chaim Nechamkin, Darryl Joyner, Joshua Fisher

# Project Statement
## Analyzing the Impact of Federal Reserve Announcement on Major Economic Indicators
In the financial markets, Federal Reserve announcements are highly anticipated events. These announcements have the power to influence investors and to trigger shifts in economic indicators. The major indicators involved are the S&P 500, Dow Jones industrial average, and the NASDAQ composite. However, this impact has been traditionally vague, and it is important to understand the specific nuances of these impacts. This is especially important when monitoring the real-time impact of these announcements. Even with the availability of text from these announcements and with financial indicators it remains an overly complex and challenging task to draw on these correlations. The goal of our project is to bridge this gap by creating a comprehensive analytical tool that will leverage natural language processing and predictive modeling techniques. This will allow us to quantify the effects of these Federal Reserve communications on the financial markets.
 
At the core of this analysis is the idea that specific language used in these Federal Reserve announcements will give critical information to the market that may shape perceptions and reactions. Language used in these announcements include the Beige Book, FOMC meeting notes, and other policy documents. By applying this advanced text analysis method, we aim to label market increases and decreases. We seek to accomplish this by extracting sentiment scores and thematic elements from these announcements. This insight will then be integrated with financial development data to develop predictive models that are capable of forecasting market movements in response to these announcements.
 
This project is designed to serve a broad audience that will include investors, policymakers, and financial analysts. We seek to provide a tool that offers deeper insights into the economic impact of these Federal Reserve decisions. To accomplish this, we seek to transfer complex financial and linguistic data into actionable predictions. As a result, our goal is to empower stakeholders to make informed decisions in a volatile market environment that is fueled by the sentiment of these Federal Reserve announcements.
 
In summary, our project addresses the need to improve the nuanced understanding of the relationship between Federal Reserve announcements and market behavior. By offering this data-driven approach to predict and interpret the economic impact of these announcements we can provide a valuable tool to analysts that are observing this related market activity.

# Background
## FOMC:

The Federal Open Market Committee (FOMC) meets eight times a year. During these meetings, members of the Federal Reserve discuss the current state of the economy and set the target for the federal funds rate. (What is the [Federal Funds Rate](https://www.investopedia.com/terms/f/federalfundsrate.asp)?). The Federal Funds Rate has significant implications for the economy, as it influences the cost of borrowing money, inflation, unemployment, and more. Due to its importance, unexpected decisions at the FOMC meetings can lead to substantial market swings.
## Beige Book:

Officially known as the “Summary of Commentary on Current Economic Conditions,” the Beige Books allows for each of the twelve Federal Reserve Districts to providing a regional perspective on economic conditions. The Beige Book is released two weeks prior to and is used by the Federal Open Market Committee (FOMC) as one of several inputs to inform their monetary policy decisions.

## Hypothesis
Trying to predict the federal funds rate before the FOMC is a common financial goal that yield exceptional returns with minimal risk if done accurately. Given the potential for profit, many firms spend countless hours building predictive models and that competition prices in the expected federal funds rate. Therefore, to profit from this situation, you must develop a better predictive model than the market average expectation.

Predictive models are most easily developed using quantitative data. For example, using consumer spending and inflation data to predict the target federal funds rate using a linear regression. Using qualitative data to develop a predictive model is much more complex meaning there may be less competition and it may be possible to predict market surprise - the market's expectation versus actual.

In [12]:
from IPython.display import IFrame
file_path = 'embed/sp500_fomc_bb_viz.html'
IFrame(file_path, width='100%', height=600)



# Methodology
To predict the market surprise, we developed a model that took text data as input and output a float that which is the predicted percentage market surprise. Text regression networks are not common and so we ended up developing our own. The backbone of this network was a Bert Sentiment Analysis model that was pre-trained on financial data. The output of this Bert portion was a (13, 3) tensor where the thirteen corresponds to the twelve FED districts plus national summary and 3 corresponds to the sentiment scores (negative, neutral, and positive). We then ran this (13, 3) tensor through two convolutional blocks: one which integrates sentiments scores across all districts and one sentiment score and one which integrates values across one district and all sentiment scores . The output of these convolutional block, (1, 3) and (13, 1), were then matrix multiplied to produce a new (13, 3) which was the integrated output. This integrated output was then combined with the output of a fully connected network which took the raw (13, 3) Bert output as input, and reduced to a single value: the predicted percentage market surprise.

![Bert_regressor_sin_bert](embed/bert_regressor_sin_bert.png "Bert Regressor w/o Bert")

# Evaluation
## Strategy
The most recent 10% of data was used for testing, the next most recent 10% was used for validation, and the remaining data was used for training. The model was evaluated using a variety of methods: including R-squared (R2), mean square error (MSE), and root mean-square error (RMSE). Furthermore, the actual surprise and predicted surprise were binned into market increase or decrease and a confusion matrix was used to assess model performance. Success was defined to be anything better than random guessing.

## Results
| Dataset | R2 | MSE | RMSE |
| ----- | ----- | ----- | ----- |
| train |  0.6952 |  0.0002 |  0.0128 |
| val | -0.5282 |  0.0007 |  0.0273 |
| test | -1.3099 |  0.0012 |  0.0344 |

Interpretation of the table above is as follows: it appears that the model did a decent job learning during training. The training R2 score of 0.6952 indicates that the model was able to explain roughly 70% of the variance in the data, which is quite excellent given the difficulty of the task. However, it does not seem that the model was able to generalize that knowledge, indicating that the model was memorizing the training set or overfitting. It is very likely all the info in the beige book, that correlates to a change in market prices, has been priced in already. In this scenario, we would expect to see the model's outputs appear similar to random chance. 

![test confusion matrix](embed/test_confusion_matrix.png "")

The confusion matrix (above) appears to confirm this theory. When we bin the actual and predicted market response (* see appendix *) and visualize the confusion matrix we can see that the model's outputs are no better than random guessing 

## Interpreting the Black Box

Deep learning methods have been called a black box because of their uninterpretable nature. However, this model design allows for some interpretation because of the convolutional block. Specifically, the (13, 1) convolutional block has a one-to-one relationship with the thirteen districts and the (1, 3) convolutional block has a one-to-one relationship with the sentiment scores. Therefore, we can matrix multiply those convolutional blocks together to get a (13, 3) array which gives an intermediary view of the AI's decision-making process. 

![param heatmap](embed/param_heatmap.png "")

Because this heatmap is from an intermediary step of the deep learning framework, we cannot interpret these parameter weights as a one-to-one relationship with districts and sentiment scores. However, we can look at what the AI is weighing as most important at this step. The AI seems to be keying in on the beige book portions from Cleveland, Kansas City, the National Summary, and San Francisco.

# Broader Impacts
Our project aims to predict the impact of Federal Reserve announcements on major economic indicators. This project holds significant implications for a wide range of stakeholders. Several individuals stand to benefit from this analytical tool including investors, policymakers, financial analysts, and economist. By utilizing this tool these stakeholders are able to anticipate market reactions to Federal Reserve communications. We seek to empower these groups to make informed decisions that will lead to profitable investments and more stable and predictable financial markets. As these stakeholders utilize this predictive capability it will help in risk management. As a result, investors are able to better hedge against potential market downturns or to profit from expected upswings.
 
Our analysis offers value to these policymakers by providing feedback on how their Federal Reserve communications are interpreted by the market. This could lead to a more effective and transparent monetary policy coming from the Federal Reserve. As these policymakers understand the nuances of how the market will react it can impact the specific language that is used and these announcements. This will also guide future communication strategies and enable a clearer and more predictable standard of guidance for these economic indicators.
 
However, this project also raises important ethical considerations. One concern is over the potential misuse of the predictive models that we develop. For example, if this tool was used by a select group of individuals or institutions this could exasperate existing inequalities in the financial markets. This is particularly true if such individuals or institutions have substantial financial resources to move these financial markets. This ability to predict market movements based on the federal reserve's announcement could lead to an unfair advantage for these individuals. Those with access to this advanced predictive tool may exploit these insights at the expense of their less informed counterparts.
 
Additionally, there is also an ethical question of transparency. The Federal Reserve seeks to manage the economy in a way that benefits the broader public. Their communications are intended to be openly interpreted. If predictive models like ours become widely used this may create pressure for the Federal Reserve to selectively alter the way that they communicate. This could lead to less transparency and more strategic messaging on the part of the Federal Reserve. In conclusion, our project offers substantial benefits. This includes enhancing decision-making processes for investors and policymakers. This tool also requires careful consideration of its broader impact. This will ensure that the tool we developed is used ethically and will not contribute to greater market inequalities.


# Visuals
your report includes at least 3 original, legible, data visualizations. Visuals are explained and integrated in the body of the report.

# Context
Our project will build upon an extensive body of existing research and methodologies that have explored the relationship between Federal Reserve announcements and financial market reactions. There have been several key studies in sources that have informed our approach. These have provided both theoretical foundations and practical insights.
 
The first study is Niemira and Klein’s (1994) Work on the Beige Book as a forecasting tool. This work underscored the significance of Federal Reserve communications in influencing market behavior. Their research highlighted how qualitative information such as that found in the Beige Book can provide early signals of economic trends. This was later reflected in they are analysis of quantitative data. The study gave us insight into the use of natural language processing to extract sentiment and key insights from these Federal Reserve documents.

 The second work is Bernanke and Kuttner’s (2005) seminole paper on the stock markets response to Federal Reserve policy actions. This study was crucial to our understanding of the mechanisms through which monetary policy can impact financial markets. Their findings on the sensitivity of stock prices to unexpected changes in the federal funds rate helped to shape our predictive modeling efforts. We were particularly interested in assessing the immediate impact of reactions to FOMC announcements.
 
Finally, the blog post by D’Amico and King (2021) On the federal reserve's communication strategies gave a contemporary perspective on how language used by the Fed can shape market expectations and behaviors. Their analysis on this topic gave great insight into the feds evolving communication style. This was especially true during times of economic uncertainty. This aligned with our focus on text and sentiment analysis to capture market sentiment and response to different types of announcements.
 
These references gave great aid along with others that were integrated into our research. These studies provide us with a great foundation and a necessary context for our project. They established the relevance of analyzing Federal Reserve announcements and validated the use of such tools as natural language processing for sentiment analysis. These tools also highlighted the importance of predictive modeling to deepen our understanding of their impact on the financial markets. And R report all references are formatted in APA style to ensure consistency and clarity in citation.
 


The above graph depicts the daily closing price of the SP500 since January 1, 1970. There are optional tick marks that show the dates of release for the Beige Books then the date of the FOMC. The tick marks for the beige books are beige and when you hover over a tick it will show the key phrases associated with that beige book for each district. The tick marks for the FOMC are green or red, they are green if the five day average closing price after the FOMC date is higher than the five day average closing price prior to the FOMC, hovering over a green/red tick will show the percentage change in the market between the five days prior and five day after the FOMC.